### Import Requirements

In [40]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import string
import random

import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aiju2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Exploration

In [2]:
# Load Office Product Amazon Reviews as pandasDF
def parse(path):
    g = gzip.open(path, "rb")
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient="index")

data_path = "../Office_Products_5.json.gz"
df = getDF(data_path)

In [3]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"11 7, 2017",A2NIJTYWADLK57,0140503528,{'Format:': ' Board book'},cotton clay,kids like story BUT while i really wanted a bo...,"good story, small size book though",1510012800,NaN,NaN
1,4.0,True,"03 7, 2017",A2827D8EEURMP4,0140503528,{'Format:': ' Hardcover'},emankcin,Bought this used and it came in great conditio...,Good,1488844800,NaN,NaN
2,5.0,True,"06 25, 2016",APB6087F4J09J,0140503528,{'Format:': ' Board book'},Starbucks Fan,Every story and book about Corduroy is Fantast...,Best Books for All Children,1466812800,NaN,NaN
3,5.0,True,"02 21, 2016",A2DHERRZIPFU7X,0140503528,{'Format:': ' Paperback'},Caitlyn Jacobson,I purchased this book for my first grade class...,Great for Math!,1456012800,NaN,NaN
4,5.0,False,"08 2, 2015",A2XCLJRGFANRC,0140503528,{'Format:': ' Hardcover'},E. Ervin,Having spent numerous years in an elementary s...,Love Corduroy,1438473600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
800352,5.0,True,"09 9, 2017",ACHG5QICJCS1F,B01HHLLLIO,{'Size:': ' 6 Rolls'},Anky,Delivered on time and is as expected.,5 stars,1504915200,NaN,NaN
800353,5.0,True,"06 2, 2017",A1YB5J8SF6QX3L,B01HHLLLIO,{'Size:': ' 6 Rolls'},DM,worked great.,good stuff,1496361600,NaN,NaN
800354,5.0,True,"04 29, 2017",A3N4O7L4HBYJX4,B01HHLLLIO,{'Size:': ' 6 Rolls'},Verdant Treasures,I used to score free UPS 4x6 labels. The blac...,Couldn't be happier with these labels,1493424000,NaN,NaN
800355,5.0,True,"02 21, 2017",A2DR3CXSWWLYXY,B01HHLLLIO,{'Size:': ' 6 Rolls'},C. F.,great,Five Stars,1487635200,NaN,NaN


This Dataset is an updated version of the Amazon review dataset released in 2014. As in the previous version, this dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs). In addition, this version provides the following features:

More reviews:
The total number of reviews is 233.1 million (142.8 million in 2014).
Newer reviews:
Current data includes reviews in the range May 1996 - Oct 2018.
Metadata:
We have added transaction metadata for each review shown on the review page. Such information includes:
Product information, e.g. color (white or black), size (large or small), package type (hardcover or electronics), etc.
Product images that are taken after the user received the product.
Added more detailed metadata of the product landing page. Such detailed information includes:
Bullet-point descriptions under product title.
Technical details table (attribute-value pairs).
Similar products table.
More categories:
Includes 5 new product categories.

Source: https://nijianmo.github.io/amazon/index.html#code

### Pre-processing

In [4]:
# a. Randomly select 500-1000 reviews from your dataset and perform steps b through d.
n_samples = random.randint(500, 1000)
review_samples = df.sample(n=n_samples)
review_samples

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
746104,5.0,True,"01 11, 2018",A2O5190IGXSN6C,B00006ICG0,{'Size:': ' 1 PACK'},Penny,Works Great! Sturdy. Nice to get something sub...,Five Stars,1515628800,NaN,NaN
432160,5.0,True,"02 25, 2015",A25IGRP8M03U1B,B0058M2YLS,{'Size:': ' 1 1/2&quot; x 1 1/2&quot;'},Francisco Coello,great!,Five Stars,1424822400,NaN,NaN
242727,5.0,True,"04 28, 2017",AN4TWQS0LRAH3,B001603YXI,"{'Color:': ' ss-cc-black', 'Package Type:': ' ...",Mags,"I've always loved these pens, they write well ...",I've always loved these pens,1493337600,NaN,NaN
12930,5.0,True,"10 20, 2008",A3ARLFYMW4QNNX,B00002NDRT,NaN,Amazon Customer,I HAVE USED THIS PRODUCT FOR YEARS AND IT WAS ...,GREAT PRODUCT FOR ORGANIZING YOUR LIFE,1224460800,NaN,NaN
578692,1.0,False,"03 8, 2017",A6WILXTKP77L0,B00JXLGESY,{'Style:': ' Printer'},J E T III,"I am never, ever, buying anything with the nam...",it burns through ink like an arsonist through ...,1488931200,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
290261,3.0,True,"03 15, 2013",A1TGX98768Z3Y2,B0016004PE,{'Size:': ' Each'},SouthernJill,"These are nice, large sized and professional l...",Nice product,1363305600,NaN,NaN
575671,5.0,True,"06 15, 2016",A393PHX1YCW0B,B00JKMVI50,{'Color:': ' Red'},Alexander P.,I was looking for some inexpensive gifts for a...,Exquisite for such an inexpensive pen.,1465948800,3,NaN
221840,5.0,True,"07 8, 2016",ANLKE2XTH3A8E,B000ZLYNH0,NaN,Jen O.,Ordered this set of Sharpies after purchasing ...,Sharpie fun with a click..,1467936000,NaN,NaN
448269,5.0,True,"10 2, 2013",A20NIKEGI7H7G6,B005VC8CG6,{'Style Name:': ' Canon CL-241XL Color Ink Car...,Guy,These cartridges print nice and seem to have a...,Canon Ink,1380672000,NaN,NaN


In [5]:
#b. Label your data based on the value of “rating of the product” i.e. as follows:
    #i. Ratings 4,5: Positive
    #ii. Rating 3: Neutral
    #iii. Ratings 1,2: Negative

# Create new column
review_samples.insert(1, "sentiment", np.nan)
review_samples

,overall,sentiment,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
746104,5.0,NaN,True,"01 11, 2018",A2O5190IGXSN6C,B00006ICG0,{'Size:': ' 1 PACK'},Penny,Works Great! Sturdy. Nice to get something sub...,Five Stars,1515628800,NaN,NaN
432160,5.0,NaN,True,"02 25, 2015",A25IGRP8M03U1B,B0058M2YLS,{'Size:': ' 1 1/2&quot; x 1 1/2&quot;'},Francisco Coello,great!,Five Stars,1424822400,NaN,NaN
242727,5.0,NaN,True,"04 28, 2017",AN4TWQS0LRAH3,B001603YXI,"{'Color:': ' ss-cc-black', 'Package Type:': ' ...",Mags,"I've always loved these pens, they write well ...",I've always loved these pens,1493337600,NaN,NaN
12930,5.0,NaN,True,"10 20, 2008",A3ARLFYMW4QNNX,B00002NDRT,NaN,Amazon Customer,I HAVE USED THIS PRODUCT FOR YEARS AND IT WAS ...,GREAT PRODUCT FOR ORGANIZING YOUR LIFE,1224460800,NaN,NaN
578692,1.0,NaN,False,"03 8, 2017",A6WILXTKP77L0,B00JXLGESY,{'Style:': ' Printer'},J E T III,"I am never, ever, buying anything with the nam...",it burns through ink like an arsonist through ...,1488931200,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290261,3.0,NaN,True,"03 15, 2013",A1TGX98768Z3Y2,B0016004PE,{'Size:': ' Each'},SouthernJill,"These are nice, large sized and professional l...",Nice product,1363305600,NaN,NaN
575671,5.0,NaN,True,"06 15, 2016",A393PHX1YCW0B,B00JKMVI50,{'Color:': ' Red'},Alexander P.,I was looking for some inexpensive gifts for a...,Exquisite for such an inexpensive pen.,1465948800,3,NaN
221840,5.0,NaN,True,"07 8, 2016",ANLKE2XTH3A8E,B000ZLYNH0,NaN,Jen O.,Ordered this set of Sharpies after purchasing ...,Sharpie fun with a click..,1467936000,NaN,NaN
448269,5.0,NaN,True,"10 2, 2013",A20NIKEGI7H7G6,B005VC8CG6,{'Style Name:': ' Canon CL-241XL Color Ink Car...,Guy,These cartridges print nice and seem to have a...,Canon Ink,1380672000,NaN,NaN


In [6]:
# Label the sentimental values
for index, row in review_samples.iterrows():
    #i. Ratings 4,5: Positive
    if (row.overall >= 4.):
        print(index, "positive")
        review_samples.loc[index, ["sentiment"]] = "positive"
    #ii. Rating 3: Neutral
    elif (row.overall < 4. or row.overall > 2.):
        print(index, "neutral")
        review_samples.loc[index, ["sentiment"]] = "neutral"
    #iii. Ratings 1,2: Negative
    elif (row.overall <= 2.):
        print(index, "negative")
        review_samples.loc[index, ["sentiment"]] = "negative"

746104 positive
432160 positive
242727 positive
12930 positive
578692 neutral
519892 positive
182444 positive
145236 positive
465863 positive
140470 positive
220362 positive
219974 positive
197482 positive
526602 neutral
638165 positive
6322 positive
241343 positive
268137 positive
213302 positive
766348 positive
305261 positive
187242 positive
734783 positive
296188 neutral
443546 positive
401983 positive
674472 positive
279640 positive
568888 neutral
234535 positive
382778 positive
459791 positive
752038 positive
512143 positive
391101 positive
418713 positive
512178 positive
718649 positive
432263 positive
81602 positive
324572 positive
632236 positive
362453 positive
714356 positive
132602 positive
718798 positive
49447 positive
143205 positive
252033 positive
425845 positive
424982 positive
107894 positive
268090 positive
421289 positive
324330 positive
203341 positive
605925 positive
173005 positive
29286 positive
773535 positive
52645 positive
137786 positive
654631 positive
684

302846 positive
130474 positive
87948 positive
441978 positive
285309 positive
252354 positive
227961 positive
566316 positive
218942 positive
537280 positive
312186 positive
437034 positive
36936 positive
365075 positive
500886 positive
253742 positive
288348 positive
770328 positive
142308 neutral
420160 positive
341640 positive
700039 positive
363104 positive
295308 positive
569993 positive
389066 positive
568108 positive
787349 positive
417358 positive
430173 neutral
352788 positive
247702 positive
102385 positive
184010 positive
270231 positive
206710 positive
156685 positive
116781 positive
418009 positive
389758 neutral
687738 positive
621461 positive
522419 neutral
354082 positive
500544 positive
47022 positive
337074 positive
540343 neutral
132008 positive
738543 positive
346550 positive
588242 positive
602147 positive
236838 neutral
30965 positive
157183 positive
628167 positive
742222 positive
745763 neutral
603669 positive
641344 positive
726475 positive
695966 positive
511

In [7]:
review_samples["sentiment"].isnull().any()

False

In [8]:
review_samples["reviewText"].isnull().any()

False

In [9]:
#c. Chose the appropriate columns for your sentiment analyzer. (Give this some thought)
sent_col = "sentiment"
review_samples[sent_col]

746104    positive
432160    positive
242727    positive
12930     positive
578692     neutral
            ...   
290261     neutral
575671    positive
221840    positive
448269    positive
479055    positive
Name: sentiment, Length: 812, dtype: object

#### Pre-process Text

In [38]:
stopwords = nltk.corpus.stopwords.words("english")

for i, text in enumerate(review_samples["reviewText"]):
    # Lowercasing
    text = text.lower()
    # Remove Digits
    text = re.sub(r'\d+','', text)
    # Remove Punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = " ".join([token for token in text.split() if (token not in stopwords)])
    # Remove trailing whitespaces
    text = ' '.join([token for token in text.split()])
    # Tokenization
    text = word_tokenize(text)    
    
    # Save to DataFrame
    review_samples["reviewText"].iloc[i] = text
    print(review_samples["reviewText"].iloc[i])

C:\Users\aiju2\AppData\Local\Temp\ipykernel_22520\3274577607.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_samples["reviewText"].iloc[i] = text


['works', 'great', 'sturdy', 'nice', 'get', 'something', 'substantial', 'money']
['great']
['ive', 'always', 'loved', 'pens', 'write', 'well', 'feel', 'great', 'hand', 'good', 'quality', 'stainless', 'steel', 'pens', 'else', 'could', 'ask']
['used', 'product', 'years', 'great', 'getting', 'discount', 'vendor', 'service', 'prompt']
['never', 'ever', 'buying', 'anything', 'name', 'epson', 'attached', 'know', 'finally', 'get', 'thing', 'print', 'even', 'somewhat', 'correctly', 'burns', 'ink', 'like', 'arsonist', 'gasoline', 'also', 'reason', 'uses', 'color', 'print', 'black', 'white', 'printer', 'decides', 'one', 'colors', 'low', 'can', 'not', 'print', 'dreadful', 'customer', 'service', 'support', 'page', 'clearly', 'designed', 'convenience', 'customers', 'never', 'buy', 'epson']
['rings', 'dont', 'stay', 'well', 'closed', 'wish', 'better', 'binders']
['love', 'pens', 'use', 'mostly', 'planner']
['labels', 'great', 'ship', 'second', 'time', 'purchase', 'regret', 'one', 'bit', 'go', 'great

['works', 'could', 'louder']
['awesome', 'ring', 'hard', 'find', 'stores', 'getting']
['nice', 'little', 'box', 'use', 'store', 'medicine', 'pill', 'bottles', 'box', 'better', 'quality', 'expecting', 'based', 'stock', 'photos', 'got', 'black', 'black', 'color', 'looks', 'great', 'even', 'though', 'came', 'scuffs', 'combination', 'easy', 'set', 'problems', 'box', 'pretty', 'small', 'dont', 'expect', 'fit', 'anything', 'big', 'inside', 'big', 'enough', 'fit', 'medium', 'sized', 'hand']
['dont', 'know', 'paper', 'printer', 'epson', 'photo', 'stylus', 'seems', 'like', 'pics', 'easily', 'scratched', 'compared', 'premium', 'presentation', 'paper', 'matte', 'ive', 'used', 'past']
['great', 'product']
['love']
['removes', 'walls', 'without', 'damaging', 'paint', 'strong', 'well']
['nice', 'way', 'keep', 'bulletin', 'borders', 'organized', 'wrinklefree']
['grandkids', 'loveem']
['nice', 'envelopes', 'easiertouse', 'sticky', 'tab', 'versus', 'complicated', 'clasp', 'design', 'item', 'good', 'pri

['works']
['mouse', 'pad', 'functions', 'utilize', 'adhesive']
['works', 'supposed', 'using', 'many', 'years', 'issues', 'far', 'epson', 'ribbon']
['product', 'described', 'arrived', 'good', 'condition', 'works', 'designed']
['expected', 'good', 'magnet']
['carpal', 'tunnel', 'really', 'bad', 'using', 'mouse', 'really', 'make', 'fingers', 'go', 'numb', 'something', 'angled', 'seems', 'help', 'micky', 'mouse', 'wrist', 'pad', 'though', 'theres', 'nothing', 'put', 'wrist', 'onexcept', 'hard', 'edge', 'love', 'concept', 'need', 'add', 'soft', 'wrist', 'pad']
['applicable']
['really', 'snap', 'set', 'easier', 'ever', 'login', 'using', 'google', 'username', 'password', 'make', 'google', 'voice', 'default', 'added', 'service', 'dollars', 'year', 'nothing', 'else', 'pay', 'taxes', 'service', 'fees', 'long', 'call', 'united', 'states', 'canada', 'wont', 'costs', 'one', 'penny']
['wife', 'landscape', 'design', 'worked', 'well']
['visited', 'many', 'office', 'supply', 'stores', 'checked', 'type'

['work', 'blame', 'seller', 'tried', 'past', 'use', 'remanufactured', 'cartridges', 'old', 'printer', 'never', 'work', 'error', 'problem']
['great']
['well', 'made', 'comfortable']
['worked', 'great', 'needed']
['shredder', 'couple', 'months', 'issues', 'happy']
['works', 'great']
['purchased', 'without', 'problems', 'time', 'package', 'paper', 'torn', 'open', 'ends', 'sealed', 'amazon', 'tape', 'would', 'okay', 'except', 'third', 'papers', 'damaged', 'since', 'obviously', 'presentation', 'paper', 'cant', 'use', 'damaged', 'worth', 'trouble', 'return', 'need', 'print', 'items', 'today', 'disappointing']
['horrible', 'wont', 'stay', 'wall', 'returned']
['love', 'pens', 'im', 'glad', 'package', 'use', 'alot', 'recommend', 'adult', 'coloring']
['exactly', 'wanted']
['surprised', 'good', 'quality']
['great', 'items', 'expected']
['perfect', 'organizer']
['like', 'dont', 'love', 'yes', 'hold', 'prisma', 'colored', 'pencils', 'varying', 'size', 'okay', 'mine', 'slips', 'around', 'could', 'de

...

['excellent']
['expected']
['works', 'larger', 'rubberband', 'guns', 'rubberband', 'gun', 'ammo', 'works', 'better']
['perfect']
['great', 'sharpie', 'markers', 'happy', 'colors']
['works', 'expected', 'good', 'quality']
['things', 'could', 'little', 'longer', 'opinion', 'look', 'longer', 'picture', 'looking', 'tie', 'together', 'two', 'ends', 'zipper', 'suitcase', 'things', 'barely', 'made', 'gap', 'seems', 'solidly', 'made', 'snaps', 'together', 'really', 'well']
['affordable', 'prices', 'fast', 'shipping', 'works', 'great', 'bringing', 'groceries', 'house']
['perfect']
['great', 'lifesaver', 'youre', 'like', 'constantly', 'trying', 'remember', 'put', 'ruler', 'last', 'project', 'limited', 'omnigrid', 'fit', 'standard', 'rulers']
['great']
['actually', 'bought', 'send', 'school', 'kindergartener', 'list', 'best', 'price', 'could', 'find', 'complaints', 'brand', 'teachers', 'always', 'request', 'reliable']
['worked', 'better', 'thought', 'sturdy', 'seem', 'ruin', 'pages']
['little', '

['exactly', 'advertised', 'shipped', 'rapidly']
['ive', 'used', 'fujitsu', 'scansnap', 'months', 'scanner', 'compare', 'kodak', 'small', 'home', 'office', 'use', 'scansnap', 'tested', 'kodak', 'business', 'personal', 'use', 'kodak', 'scanner', 'speed', 'really', 'churns', 'images', 'scan', 'lot', 'every', 'day', 'may', 'good', 'buy', 'dont', 'mind', 'lack', 'good', 'userfriendly', 'software', 'comes', 'software', 'plus', 'thirdparty', 'software', 'paperport', 'omnipage', 'none', 'measure', 'intuitive', 'userfriendly', 'software', 'fujitsu', 'scansnap', 'tweaked', 'kodak', 'settings', 'hours', 'still', 'performing', 'well', 'scansnap', 'box', 'example', 'scansnap', 'automatically', 'adjusts', 'size', 'receipt', 'scans', 'saves', 'whats', 'needed', 'even', 'tweaking', 'kodak', 'doesnt', 'scansnap', 'organizer', 'software', 'much', 'intuitive', 'kodak', 'software', 'making', 'saving', 'filing', 'significantly', 'easier', 'scanning', 'half', 'battle', 'able', 'find', 'later', 'important', 

['use', 'daily', 'homework', 'worth', 'price']
['looking', 'lot', 'products', 'tilt', 'hp', 'laptop', 'finally', 'settled', 'one', 'wasnt', 'disappointed', 'near', 'perfect', 'light', 'weight', 'keeps', 'laptop', 'cool', 'day', 'lots', 'angle', 'adjustments', 'fits', 'particular', 'laptop', 'beautifully', 'although', 'follow', 'advice', 'previous', 'buyer', 'put', 'cardboard', 'strips', 'come', 'packaging', 'front', 'stand', 'slightly', 'raise', 'laptop', 'level', 'stands', 'two', 'front', 'tabs', 'make', 'much', 'comfortable', 'rest', 'hands', 'laptop']
['pictured', 'cute']
['always', 'expo', 'never', 'wrong', 'love', 'product', 'great', 'quality']
['husband', 'moved', 'new', 'home', 'last', 'year', 'extra', 'living', 'room', 'able', 'use', 'office', 'bought', 'new', 'office', 'furniture', 'mission', 'get', 'everything', 'neat', 'organized', 'could', 'desk', 'bought', 'nice', 'file', 'cabinet', 'built', 'needed', 'file', 'holders', 'put', 'similar', 'file', 'dividers', 'office', 'work

['im', 'artist', 'lot', 'pencil', 'work', 'little', 'eraser', 'lets', 'precision', 'erasing', 'without', 'disturbing', 'rest', 'drawing', 'great']
['nothing', 'takes', 'back', 'elementary', 'school', 'smelly', 'markers', 'classic', 'scents', 'remember', 'watermelon', 'cherry', 'nasty', 'black', 'licorice']
['love', 'love', 'theyre', 'made', 'america', 'soft', 'cover', 'perfect', 'notetaking', 'writing', 'etc', 'highly', 'recommended']
['datahookproductlinklinked', 'classalinknormal', 'hrefeasyreadregistercheckbooktransactionregisterscalendarspackofdpbyqcgkrefcmcrarpdrvwtxtieutfeasy', 'read', 'register', 'checkbook', 'transaction', 'registers', 'calendars', 'pack', 'write', 'checks', 'month', 'using', 'debit', 'card', 'still', 'need', 'place', 'write', 'transactions', 'keep', 'much', 'money', 'spending', 'constantly', 'running', 'checkbook', 'registers', 'annoying', 'decided', 'purchase', 'get', 'one', 'register', 'order', 'checks', 'received', 'quickly', 'fit', 'check', 'book', 'perfec

['regular', 'size', 'ones']
['good', 'ink']
['product', 'arrived', 'wrapped', 'thin', 'clear', 'plastic', 'wrapping', 'colorful', 'packaging', 'boxes', 'labels', 'appreciate', 'cost', 'cutting', 'efforts', 'pass', 'savings', 'consumers', 'edges', 'papers', 'wrinkledbentcrushed', 'due', 'lack', 'solid', 'box', 'thick', 'plastic', 'hold', 'edges', 'luckily', 'minor', 'problem', 'impact', 'performance', 'product', 'labels', 'printed', 'clearly', 'easy', 'apply', 'havent', 'found', 'surface', 'didnt', 'stick', 'however', 'seem', 'ever', 'slightly', 'less', 'sticky', 'major', 'name', 'brand', 'applying', 'plastic', 'bags', 'strongly', 'recommended']
['quick', 'frugal', 'printer', 'easy', 'setup', 'works', 'without', 'issues', 'surprised', 'fast', 'first', 'page', 'spits', 'good', 'quality', 'print']
['great', 'paper', 'used', 'kind', 'printer', 'time', 'paper', 'dont', 'get', 'hung', 'machine', 'printer', 'fussy', 'printer', 'looks', 'good', 'printed', 'buy', 'needed', 'amazon']
['nice', 'l

In [42]:
# Stemming
stemmer = PorterStemmer()
stemmed_samples = review_samples.copy()

for i, text in enumerate(stemmed_samples["reviewText"]):
    text = [stemmer.stem(word) for word in text]
    
    # Save to DataFrame
    stemmed_samples["reviewText"].iloc[i] = text
    print(stemmed_samples["reviewText"].iloc[i])

C:\Users\aiju2\AppData\Local\Temp\ipykernel_22520\2648333906.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stemmed_samples["reviewText"].iloc[i] = text


['work', 'great', 'sturdi', 'nice', 'get', 'someth', 'substanti', 'money']
['great']
['ive', 'alway', 'love', 'pen', 'write', 'well', 'feel', 'great', 'hand', 'good', 'qualiti', 'stainless', 'steel', 'pen', 'els', 'could', 'ask']
['use', 'product', 'year', 'great', 'get', 'discount', 'vendor', 'servic', 'prompt']
['never', 'ever', 'buy', 'anyth', 'name', 'epson', 'attach', 'know', 'final', 'get', 'thing', 'print', 'even', 'somewhat', 'correctli', 'burn', 'ink', 'like', 'arsonist', 'gasolin', 'also', 'reason', 'use', 'color', 'print', 'black', 'white', 'printer', 'decid', 'one', 'color', 'low', 'can', 'not', 'print', 'dread', 'custom', 'servic', 'support', 'page', 'clearli', 'design', 'conveni', 'custom', 'never', 'buy', 'epson']
['ring', 'dont', 'stay', 'well', 'close', 'wish', 'better', 'binder']
['love', 'pen', 'use', 'mostli', 'planner']
['label', 'great', 'ship', 'second', 'time', 'purchas', 'regret', 'one', 'bit', 'go', 'great', 'printer', 'make', 'sure', 'feed', 'one', 'time', 'b

['return', 'hard', 'sit', 'upon', 'could', 'feel', 'gel', 'easi', 'return']
['hold', 'secur', 'fold', 'flat', 'complaint', 'far']
['love', 'let', 'count', 'way', 'great', 'littl', 'lamin', 'paid', 'back', 'school', 'sale', 'may', 'influenc', 'rate', 'honestli', 'dont', 'know', 'could', 'better', 'heat', 'fast', 'job', 'recommend']
['im', 'type', 'person', 'write', 'neg', 'review', 'anyon', 'ive', 'disappoint', 'frustrat', 'colo', 'pencil', 'come', 'ship', 'pencil', 'halfway', 'sharpen', 'guess', 'point', 'way', 'whether', 'sharp', 'want', 'start', 'sharpen', 'sharpen', 'pencil', 'tip', 'would', 'break', 'lot', 'three', 'pencil', 'actual', 'wood', 'split', 'right', 'side', 'save', 'two', 'got', 'frustrat', 'threw', 'garbag', 'spent', 'much', 'money', 'set', 'right', 'penni', 'that', 'ive', 'artist', 'year', 'draw', 'watercolor', 'water', 'solubl', 'color', 'pencil', 'im', 'frustrat', 'coupl', 'peopl', 'tell', 'like', 'brand', 'even', 'think', 'look', 'ask', 'light', 'fast', 'test', 'too

['fellow', 'automax', 'c', 'automat', 'shred', 'stack', 'document', 'short', 'order', 'firstli', 'want', 'activ', 'compar', 'version', 'ativa', 'httpamzntognwgrp', 'pro', 'fast', 'automat', 'shred', 'stack', 'page', 'manual', 'feeder', 'hold', 'consider', 'sheet', 'time', 'fellow', 'claim', 'shred', 'paper', 'clip', 'stapl', 'cd', 'credit', 'card', 'well', 'paper', 'con', 'pricey', 'shredder', 'recommend', 'singl', 'user']
['perfect', 'problem', 'deliv', 'time', 'thank']
['realli', 'bother', 'buy', 'set', 'new', 'pencil', 'sinc', 'day', 'late', 'teen', 'earli', 'take', 'portraitur', 'class', 'local', 'artist', 'wide', 'rang', 'pencil', 'variou', 'hard', 'lead', 'receiv', 'set', 'via', 'vine', 'real', 'treat', 'mead', 'one', 'brand', 'use', 'work', 'well', 'variou', 'effect', 'type', 'draw', 'pencil', 'come', 'love', 'metal', 'case', 'protect', 'presharpen', 'issu', 'other', 'express', 'review', 'sharpen', 'inconsist', 'use', 'solid', 'brass', 'manual', 'sharpen', 'art', 'store', 'rathe

['heavi', 'solid', 'rock', 'work', 'perfectli', 'box', 'tape', 'finger', 'watch', 'sharp', 'cutter', 'edg', 'youll', 'happi']
['like', 'write', 'small', 'area', 'im', 'write', 'quit', 'bit', 'small', 'area', 'dri', 'fast', 'realli', 'like', 'retract', 'need', 'write', 'often', 'lot', 'better', 'take', 'cap', 'everi', 'time', 'want', 'write', 'someth']
['preserv', 'beauti', 'hardwood', 'floor']
['look', 'purchas', 'st', 'fountain', 'pen', 'need', 'everydayknockaroundworkhors', 'worryfre', 'look', 'doesnt', 'get', 'better', 'pilot', 'metropolitan', 'substanti', 'hand', 'write', 'smoothli', 'box', 'time', 'everi', 'time', 'drawback', 'come', 'medium', 'nib', 'closer', 'western', 'fine', 'gener', 'perfect', 'around', 'use', 'come', 'bladder', 'clean', 'may', 'use', 'convert', 'one', 'cartridg', 'may', 'refil', 'blunt', 'syring', 'bottl', 'ink', 'choic', 'may', 'also', 'purchas', 'addit', 'cartridg', 'limit', 'ink', 'choic', 'casual', 'use', 'make', 'conveni', 'excel', 'buy', 'cant', 'recom

...

['small']
['amaz', 'great', 'artist', 'decor', 'almost', 'surfac', 'think', 'embellish', 'leather', 'bag', 'canva', 'leather', 'shoe', 'possibl', 'endless', 'tip', 'great', 'control', 'cant', 'get', 'easi', 'get', 'nice', 'consist', 'line', 'oh', 'specif', 'look', 'burgandi', 'color', 'got', 'lucki', 'set', 'great', 'color', 'zentangl']
['alway', 'love', 'replac', 'cartridg', 'price', 'amaz', 'compar', 'local', 'store', 'dont', 'last', 'quit', 'long', 'brand', 'name', 'problem', 'price', 'use', 'year']
['great', 'qualiti', 'price']
['three', 'printer', 'use', 'regular', 'print', 'job', 'postag', 'print', 'envelop', 'work', 'fine', 'replac', 'toner', 'use', 'pretti', 'expens', 'printer', 'releas', 'replac', 'toner', 'overal', 'qualiti', 'printer']
['work', 'great', 'look', 'good', 'advertis']
['alway', 'happi', 'averi']
['im', 'high', 'mainten', 'come', 'pen', 'theyr', 'easi', 'hold', 'gel', 'ink', 'flow', 'smoothli', 'even', 'slick', 'credit', 'card', 'receipt', 'sign', 'checkout', 'me

['work', 'better', 'thought', 'sturdi', 'seem', 'ruin', 'page']
['littl', 'clip', 'like', 'larger', 'parent', 'work', 'well', 'print', 'read', 'later']
['cart', 'bought', 'tv', 'well', 'built', 'easi', 'assembl', 'look', 'heavi', 'duti', 'well', 'construct', 'would', 'recommend']
['swift', 'pen']
['kid', 'think', 'look', 'sexi', 'next', 'imac', 'featur', 'work', 'well', 'troubl', 'connect', 'get', 'start', 'took', 'minut', 'perfect', 'fit', 'desk', 'like', 'fold', 'sleek', 'black', 'silver', 'piec', 'art', 'use', 'issu', 'im', 'color', 'littl', 'im', 'sure', 'ill', 'work', 'take', 'time', 'read', 'manual', 'scanner', 'work', 'great', 'canon', 'quiet', 'fast', 'fine', 'year', 'later', 'still', 'love', 'printer', 'troubl', 'perfect', 'companion', 'imac']
['pencil', 'sharpen', 'becam', 'talk', 'social', 'media', 'everyon', 'tri', 'itmyself', 'includ', 'let', 'say', 'invest', 'lot', 'money', 'prismacolor', 'pencil', 'persnicketi', 'run', 'someon', 'say', 'sharpen', 'sooth', 'away', 'woo', 

['comment']
['great', 'valu', 'purchas', 'mani', 'continu']
['love', 'beati', 'color']
['great', 'qualiti', 'easi', 'use']
['oh', 'gosh', 'great', 'product', 'use', 'week', 'exactli', 'like', 'name', 'brand', 'much', 'cheaper', 'definit', 'purchas']
['order', 'april', 'back', 'replac', 'ink', 'cartridg', 'box', 'say', 'china', 'feb', 'assum', 'expir', 'date', 'well', 'print', 'bare', 'percept', 'like', 'dri', 'cartridg', 'wast', 'moneyso']
['yeah', 'babi', 'ok', 'mayb', 'littl', 'top', 'file', 'aid', 'hey', 'lifesav', 'modern', 'day', 'paper', 'avalanch', 'paperless', 'offic', 'yeah', 'right', 'less', 'paper', 'mayb', 'paper', 'home', 'offic', 'put', 'field', 'letter', 'size', 'sorter', 'realli', 'help', 'keep', 'control', 'cover', 'nice', 'look', 'durabl', 'even', 'water', 'resist', 'get', 'bit', 'grubbi', 'wipe', 'clean', 'overs', 'protect', 'record', 'still', 'small', 'enough', 'fit', 'ina', 'briefcas', 'desktop', 'eas', 'spine', 'accordion', 'style', 'plenti', 'room', 'section', 'l

['nurs', 'friend', 'love', 'gift', 'highlight', 'smoothli']
['essenti', 'compon', 'mani', 'printmak', 'collag', 'project', 'class', 'kid', 'adult']
['econom', 'ink', 'make', 'cring', 'thought', 'print', 'color', 'ink', 'still', 'produc', 'great', 'imag', 'qualiti', 'factori', 'pigment', 'drawback', 'harder', 'remov', 'print', 'factori', 'pigment', 'thumb', 'tab', 'still', 'well', 'worth', 'hassl', 'price']
['use', 'rubber', 'band', 'sort', 'thing', 'good', 'littl', 'small', 'tupperwar', 'perfect', 'x', 'card', 'thinish', 'thick']
['im', 'artist', 'never', 'use', 'type', 'pencil', 'said', 'learn', 'use', 'oil', 'base', 'guess', 'buildabl']
['impress', 'work', 'great', 'hp', 'smart', 'photo', 'printer', 'compat', 'issu', 'like', 'clear', 'see', 'complet', 'full', 'arriv', 'print', 'sheet', 'paper', 'black', 'cartridg']
['lap', 'desk', 'indispens', 'life', 'ive', 'enter', 'lap', 'desk', 'nirvana', 'product', 'thought', 'everi', 'possibl', 'thing', 'make', 'perfect', 'lap', 'desk', 'cool',

['dont', 'normal', 'write', 'review', 'card', 'exactli', 'need', 'went', 'major', 'offic', 'suppli', 'store', 'without', 'luck', 'studi', 'cpa', 'exam', 'take', 'least', 'month', 'card', 'wont', 'last', 'month', 'also', 'afraid', 'would', 'feel', 'rough', 'creat', 'massiv', 'pile', 'card', 'case', 'card', 'smooth', 'smooth', 'pen', 'marker', 'doesnt', 'take', 'much', 'room', 'one', 'suggest', 'would', 'like', 'line', 'card', 'blank', 'one', 'side', 'line', 'usual', 'like', 'question', 'blank', 'side', 'answer', 'side', 'take', 'entir', 'card', 'would', 'nice', 'line', 'keep', 'organ']
['great', 'phone']
['big', 'person', 'say', 'cushion', 'let', 'lb', 'nuff', 'said']
['cartridg', 'good', 'inexpens', 'laser', 'printer']
['aktl', 'smelli', 'im', 'disappoint', 'chemic', 'compani', 'like', 'unsavvi', 'produc', 'offens', 'odor', 'product', 'day', 'age', 'enlighten', 'offens', 'chemic', 'shown', 'ignor', 'concern', 'untrain', 'nose', 'tell', 'keyboard', 'tray', 'outgass', 'volatil', 'organ',

In [43]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_samples = review_samples.copy()

for i, text in enumerate(lemmatized_samples["reviewText"]):
    text = [stemmer.stem(word) for word in text]
    
    # Save to DataFrame
    lemmatized_samples["reviewText"].iloc[i] = text
    print(lemmatized_samples["reviewText"].iloc[i])

C:\Users\aiju2\AppData\Local\Temp\ipykernel_22520\1844177594.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lemmatized_samples["reviewText"].iloc[i] = text


['work', 'great', 'sturdi', 'nice', 'get', 'someth', 'substanti', 'money']
['great']
['ive', 'alway', 'love', 'pen', 'write', 'well', 'feel', 'great', 'hand', 'good', 'qualiti', 'stainless', 'steel', 'pen', 'els', 'could', 'ask']
['use', 'product', 'year', 'great', 'get', 'discount', 'vendor', 'servic', 'prompt']
['never', 'ever', 'buy', 'anyth', 'name', 'epson', 'attach', 'know', 'final', 'get', 'thing', 'print', 'even', 'somewhat', 'correctli', 'burn', 'ink', 'like', 'arsonist', 'gasolin', 'also', 'reason', 'use', 'color', 'print', 'black', 'white', 'printer', 'decid', 'one', 'color', 'low', 'can', 'not', 'print', 'dread', 'custom', 'servic', 'support', 'page', 'clearli', 'design', 'conveni', 'custom', 'never', 'buy', 'epson']
['ring', 'dont', 'stay', 'well', 'close', 'wish', 'better', 'binder']
['love', 'pen', 'use', 'mostli', 'planner']
['label', 'great', 'ship', 'second', 'time', 'purchas', 'regret', 'one', 'bit', 'go', 'great', 'printer', 'make', 'sure', 'feed', 'one', 'time', 'b

['hold', 'secur', 'fold', 'flat', 'complaint', 'far']
['love', 'let', 'count', 'way', 'great', 'littl', 'lamin', 'paid', 'back', 'school', 'sale', 'may', 'influenc', 'rate', 'honestli', 'dont', 'know', 'could', 'better', 'heat', 'fast', 'job', 'recommend']
['im', 'type', 'person', 'write', 'neg', 'review', 'anyon', 'ive', 'disappoint', 'frustrat', 'colo', 'pencil', 'come', 'ship', 'pencil', 'halfway', 'sharpen', 'guess', 'point', 'way', 'whether', 'sharp', 'want', 'start', 'sharpen', 'sharpen', 'pencil', 'tip', 'would', 'break', 'lot', 'three', 'pencil', 'actual', 'wood', 'split', 'right', 'side', 'save', 'two', 'got', 'frustrat', 'threw', 'garbag', 'spent', 'much', 'money', 'set', 'right', 'penni', 'that', 'ive', 'artist', 'year', 'draw', 'watercolor', 'water', 'solubl', 'color', 'pencil', 'im', 'frustrat', 'coupl', 'peopl', 'tell', 'like', 'brand', 'even', 'think', 'look', 'ask', 'light', 'fast', 'test', 'took', 'upon', 'order', 'without', 'read', 'everyth', 'known', 'would', 'probab

['epson', 'alway', 'make', 'excel', 'qualiti', 'paper']
['standard', 'binder', 'howev', 'daughter', 'favorit', 'color', 'purpl', 'love']
[]
['expect']
['let', 'start', 'say', 'full', 'awar', 'product', 'made', 'fake', 'wood', 'plastic', 'pleas', 'happi', 'mine', 'order', 'three', 'differ', 'piec', 'furinno', 'light', 'cherri', 'shelv', 'actual', 'use', 'end', 'tabl', 'arriv', 'super', 'fast', 'packag', 'well', 'set', 'super', 'easi', 'yet', 'like', 'plastic', 'shelv', 'unit', 'tube', 'screw', 'one', 'anoth', 'make', 'stay', 'togeth', 'im', 'custom', 'one', 'make', 'bit', 'higher', 'sit', 'evenli', 'reclin', 'well', 'custom', 'everi', 'shelf', 'appli', 'selfadhes', 'lamin', 'sheet', 'top', 'order', 'protect', 'wood', 'spill', 'condens', 'glass', 'ring', 'etc', 'instead', 'fiddl', 'around', 'coaster', 'face', 'everyon', 'manner', 'use', 'even', 'sit', 'plain', 'site', 'im', 'make', 'shini', 'look', 'pretti', 'cool', 'actual', 'give', 'sort', 'glow', 'bought', 'pack', 'amazon', 'like', 't

['nice', 'pen', 'like', 'outlin', 'pictur', 'fine', 'tip', 'make', 'easier', 'also', 'like', 'outlin', 'glitter', 'pen', 'fine', 'point', 'pen', 'might', 'check', 'see', 'come', 'fine', 'point']
['beauti', 'color']
['nice', 'sturdi', 'great', 'organ', 'import', 'famili', 'document']
['best', 'mechan', 'pencil', 'ever', 'design', 'grab', 'exel', 'slip']
['write', 'realli', 'nice', 'clean', 'good', 'assort', 'color', 'look', 'forward', 'get', 'artist', 'creativ', 'pen']
['alway', 'great']
['favorit', 'color', 'utensil', 'johanna', 'basford', 'color', 'book', 'super', 'fine', 'line']
['great', 'product']
['overal', 'write', 'well']
['good', 'valu']
['great', 'pen']
['great', 'valu', 'could', 'pass', 'pen', 'bic', 'good', 'qualiti']
['use']
['great']
['price', 'paid', 'scale', 'versatil', 'accuraci', 'surpris', 'ive', 'use', 'sell', 'item', 'onlin', 'weigh', 'item', 'engin', 'experi', 'refriger', 'work', 'lightweight', 'scale', 'great', 'use']
['look', 'forward', 'see', 'long', 'last', 'pe

...

['excel', 'sound', 'qualiti', 'electron', 'interfer']
['sharpen', 'good', 'use', 'quit', 'bit', 'alreadi', 'sinc', 'wait', 'use', 'woodless', 'color', 'pencil', 'sharpen', 'fine', 'point', 'need', 'tini', 'spot', 'color', 'would', 'recommend', 'anyon', 'need', 'sharpen', 'type', 'pencil', 'excel', 'product', 'star']
['love', 'pen', 'picki', 'pen', 'person', 'recent', 'discov', 'pentel', 'energel', 'pen', 'fell', 'love', 'immedi', 'ink', 'flow', 'smoothli', 'dri', 'fast', 'color', 'true', 'importantli', 'tip', 'isnt', 'scratchi', 'metal', 'barrel', 'huge', 'step', 'plastic', 'one', 'come', 'multipack', 'heavi', 'enough', 'feel', 'good', 'hand', 'even', 'without', 'rubber', 'grip', 'still', 'comfort', 'barel', 'hold', 'either', 'refil', 'person', 'use', 'needl', 'tip', 'fit', 'perfectli', 'write', 'like', 'dream']
['nice', 'sturdi', 'card', 'blackandwhit', 'imag', 'mixtur', 'blue', 'green', 'messag', 'father', 'wish', 'could', 'turn', 'back', 'time', 'basic', 'enjoy', 'son', 'younger', '

['excel']
['expect']
['work', 'larger', 'rubberband', 'gun', 'rubberband', 'gun', 'ammo', 'work', 'better']
['perfect']
['great', 'sharpi', 'marker', 'happi', 'color']
['work', 'expect', 'good', 'qualiti']
['thing', 'could', 'littl', 'longer', 'opinion', 'look', 'longer', 'pictur', 'look', 'tie', 'togeth', 'two', 'end', 'zipper', 'suitcas', 'thing', 'bare', 'made', 'gap', 'seem', 'solidli', 'made', 'snap', 'togeth', 'realli', 'well']
['afford', 'price', 'fast', 'ship', 'work', 'great', 'bring', 'groceri', 'hous']
['perfect']
['great', 'lifesav', 'your', 'like', 'constantli', 'tri', 'rememb', 'put', 'ruler', 'last', 'project', 'limit', 'omnigrid', 'fit', 'standard', 'ruler']
['great']
['actual', 'bought', 'send', 'school', 'kindergarten', 'list', 'best', 'price', 'could', 'find', 'complaint', 'brand', 'teacher', 'alway', 'request', 'reliabl']
['work', 'better', 'thought', 'sturdi', 'seem', 'ruin', 'page']
['littl', 'clip', 'like', 'larger', 'parent', 'work', 'well', 'print', 'read', 'la

['although', 'littl', 'larger', 'expect', 'well', 'made', 'use', 'green', 'color', 'assur', 'find', 'easili']
['good', 'pocket', 'buddi', 'realli', 'common', 'site', 'use', 'arent', 'mani', 'entri', 'option', 'lot', 'password', 'want', 'keep', 'love', 'size', 'thoughgreat', 'travel']
['nice', 'stapler']
['great', 'productperform', 'advertis', 'recommend']
['bit', 'overpr', 'opinion', 'far', 'good', 'three', 'particular', 'item', 'alway', 'misplac', 'far', 'realli', 'come', 'handi', 'hope', 'last']
['sturdi', 'keep', 'organ', 'space', 'saver', 'pocket', 'chart', 'hang', 'back', 'one', 'storag', 'shelv', 'front', 'shelv', 'face', 'classroom', 'student', 'access', 'materi', 'back', 'face', 'desk', 'keep', 'homework', 'quizz', 'worksheet', 'upcom', 'month', 'pocket', 'chart', 'need', 'know', 'exactli', 'grab', 'instantli', 'dont', 'clutter', 'stack', 'tray', 'banker', 'box', 'desk']
['great', 'item']
['ok', 'work']
['fit', 'good', 'great', 'coc', 'work', 'littl', 'wrinkl', 'cute', 'color',

['alway', 'expo', 'never', 'wrong', 'love', 'product', 'great', 'qualiti']
['husband', 'move', 'new', 'home', 'last', 'year', 'extra', 'live', 'room', 'abl', 'use', 'offic', 'bought', 'new', 'offic', 'furnitur', 'mission', 'get', 'everyth', 'neat', 'organ', 'could', 'desk', 'bought', 'nice', 'file', 'cabinet', 'built', 'need', 'file', 'holder', 'put', 'similar', 'file', 'divid', 'offic', 'work', 'realli', 'cheap', 'metal', 'hanger', 'alway', 'seem', 'bend', 'never', 'stay', 'straight', 'realli', 'dont', 'like', 'although', 'one', 'look', 'exactli', 'one', 'offic', 'way', 'better', 'thick', 'realli', 'heavi', 'duti', 'metal', 'hanger', 'cheap', 'fit', 'perfectli', 'file', 'cabinet', 'think', 'hold', 'long', 'time', 'appreci', 'plastic', 'label', 'holder', 'white', 'label', 'came', 'abl', 'organ', 'home', 'work', 'paperwork', 'offic', 'realli', 'help', 'lot', 'keep', 'thing', 'fluid', 'organ', 'look', 'near', 'far', 'find', 'piec', 'paper', 'everyth', 'nice', 'organ', 'tuck', 'protect', 

['innov', 'skeptic', 'actual', 'work', 'exactli', 'describ', 'manufactur', 'seller', 'thank', 'tab', 'actual', 'reus', 'time', 'time', 'im', 'sold', 'divid', 'purchas']
['good', 'tool', 'learn', 'spell', 'word', 'spell', 'test', 'spread', 'floor', 'let', 'son', 'find', 'word', 'call', 'make', 'learn', 'littl', 'fun']
['red', 'waterproof', 'give', 'good', 'color', 'ink', 'write', 'crisp', 'line', 'tad', 'dri', 'side', 'feather', 'easili', 'mani', 'ink', 'especi', 'compar', 'noodler', 'bulletproof', 'black', 'use', 'pilot', 'ef', 'nib', 'flow', 'well', 'color', 'true', 'bright', 'red', 'nonwaterproof', 'red', 'expect', 'there', 'much', 'interest', 'shade', 'ink', 'feather', 'less', 'id', 'rate', 'five', 'star', 'quit', 'practic', 'waterproof', 'red', 'ink']
['good']
['brush', 'awesom', 'work', 'water', 'solubl', 'color', 'pencil', 'small', 'theyr', 'best', 'suit', 'work', 'small', 'piec', 'tight', 'spot', 'point', 'stay', 'sharp', 'easi', 'clean', 'travel', 'well', 'easier', 'use', 'trad

In [3]:
# Plot the vector distribution (bar graph)

### Text Representation

In [69]:
review_texts = stemmed_samples["reviewText"] # Use stemmed samples
def dummy(doc):
    return doc

In [70]:
review_texts.shape

(812,)

In [71]:
# Apply TfidfVectorizer"
tfidf_vect = TfidfVectorizer(
    analyzer="word", 
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None)
X_tfidf = tfidf_vect.fit_transform(review_texts)
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

(812, 2995)
['aa', 'aaa', 'aaaaa', 'abil', 'abl', 'abox', 'absent', 'absolut', 'abund', 'abus', 'access', 'accessori', 'accid', 'accident', 'accommod', 'accordingli', 'accordion', 'accur', 'accuraci', 'achil', 'acknowledg', 'acquaint', 'across', 'activ', 'actual', 'ad', 'add', 'addict', 'addit', 'addon', 'address', 'adequ', 'adher', 'adhes', 'adjust', 'admir', 'admit', 'ador', 'adorama', 'adult', 'advanc', 'advantag', 'advertis', 'advic', 'advis', 'aeron', 'aesthet', 'affix', 'afford', 'afraid', 'afterward', 'age', 'ago', 'agre', 'ah', 'ahead', 'aid', 'aio', 'air', 'aktl', 'al', 'ala', 'alarm', 'alert', 'align', 'allinon', 'allot', 'allow', 'allstar', 'almost', 'alon', 'along', 'alot', 'alreadi', 'also', 'altern', 'alternati', 'although', 'aluminum', 'alway', 'amaz', 'amazingli', 'amazon', 'america', 'american', 'ammo', 'ammonia', 'among', 'amongst', 'amount', 'anchor', 'andor', 'android', 'andsubtract', 'angl', 'angri', 'annoy', 'annoyingli', 'anoth', 'answer', 'antimicrobi', 'antiqu'

### Modeling (Lexicon-based approach)

In [73]:
# Valence Aware Dictionary and Sentiment Reasoner (VADR)
VADR_analyzer = SentimentIntensityAnalyzer()
# Pass the analyzer for head 5 rows
for index, row in review_samples.head().iterrows():
    vs = VADR_analyzer.polarity_scores(row["reviewText"])
    full_text = " ".join(row.reviewText)
    print(
        f"Index: {index}\n"+
        f"Sentimental Analysis Result: {vs}\n"+
        f"Actual category: {row.sentiment}\n"+
        f"Full Text:\n{full_text}\n"+
        "-"*50)

Index: 746104
Sentimental Analysis Result: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Actual category: positive
Full Text:
works great sturdy nice get something substantial money
--------------------------------------------------
Index: 432160
Sentimental Analysis Result: {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6249}
Actual category: positive
Full Text:
great
--------------------------------------------------
Index: 242727
Sentimental Analysis Result: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Actual category: positive
Full Text:
ive always loved pens write well feel great hand good quality stainless steel pens else could ask
--------------------------------------------------
Index: 12930
Sentimental Analysis Result: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Actual category: positive
Full Text:
used product years great getting discount vendor service prompt
--------------------------------------------------
Index: 578692
Sentimental Analysis

### Validation